In [2]:
from main import *


from sklearn.pipeline import Pipeline
from sktime.transformations.panel.tsfresh import TSFreshFeatureExtractor
from sktime.classification.sklearn import RotationForest


from sktime.pipeline import make_pipeline
from sktime.transformations.panel.rocket import MiniRocketMultivariate
from sklearn.linear_model import LogisticRegression

from sktime.classification.interval_based import DrCIF
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.classification.dictionary_based import TemporalDictionaryEnsemble
from sktime.classification.deep_learning import InceptionTimeClassifier

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
machines = ["M01", "M02","M03"]
process_names = ["OP00","OP01","OP02","OP03","OP04","OP05","OP06","OP07","OP08","OP09","OP10","OP11","OP12","OP13","OP14"]
labels = ["good","bad"]
path_to_dataset = Path("./data/").absolute()

In [4]:
X_data = []
y_data = []

for process_name, machine, label in itertools.product(process_names, machines, labels):
    data_path = os.path.join(path_to_dataset, machine, process_name, label)
    data_list, data_label = data_loader_utils.load_tool_research_data(data_path, label=label, add_additional_label = True, verbose = False)
    X_data.extend(data_list)
    y_data.extend(data_label)

df = get_df(X_data, y_data)

In [ ]:
rotf = RotationForest(n_jobs = -1) 

param_grid_rotf = {
    "n_estimators": [50, 200]
    }

results_rotf, objects_rotf = get_uni_cv_results(rotf, param_grid_rotf, df)

with open("cv_results/rotf.pkl", "wb") as f: 
    pickle.dump(results_rotf, f)

In [ ]:
fp = Pipeline([
    (
        "transform",
        TSFreshFeatureExtractor(
            default_fc_parameters="efficient",
            show_warnings = False),
    ),
    ("classifier", RotationForest ()),
])

param_grid_fp = {
    "classifier__n_estimators": [50, 200]
    }

cv_results_fp, gs_objects_fp = get_cv_results(fp, param_grid_fp, df)

with open("cv_results/fp.pkl", "wb") as f:
    pickle.dump(cv_results_fp, f)

In [ ]:
drcif = DrCIF(time_limit_in_minutes = 10, n_jobs = -1)

param_grid_drcif = {
    "att_subsample_size": [5, 10], # 10 is default and 5 is half of the default
}

cv_results_drcif, gs_objects_drcif = get_cv_results(drcif, param_grid_drcif, df)

with open("cv_results/drcif.pkl", "wb") as f:
    pickle.dump(cv_results_drcif, f)

In [7]:
trf = RandomShapeletTransform(time_limit_in_minutes = 10, n_jobs = -1)
clf = RotationForest(n_jobs = -1)

stc = Pipeline([("transform", trf), ("classifier", clf)])

param_grid_stc = {
    "transform__n_shapelet_samples": [1000],
    "classifier__n_estimators": [50, 200]
} 

cv_results_stc, gs_objects_stc = get_cv_results(stc, param_grid_stc, df)

with open ("cv_results/stc.pkl", "wb") as f:
    pickle.dump(cv_results_stc, f)

Hyperparameter tuning on machine-wise split...
Hyperparameter tuning on time-wise split...
Hyperparameter tuning on operation-wise split...


In [5]:
tde = TemporalDictionaryEnsemble(max_win_len_prop= 0.5, time_limit_in_minutes = 10, contract_max_n_parameter_samples = 100, n_jobs = 1) # To reduce memory overhead

param_grid_tde = {
    "max_ensemble_size" : [5, 25],
    "randomly_selected_params" : [5, 25]
}

cv_results_tde, gs_objects_tde = get_cv_results(tde, param_grid_tde, df)

with open("cv_results/tde.pkl", "wb") as f: 
    pickle.dump(cv_results_tde, f)

Hyperparameter tuning on machine-wise split...
Hyperparameter tuning on time-wise split...
Hyperparameter tuning on operation-wise split...


In [ ]:
trf = MiniRocketMultivariate(n_jobs = 1) 
clf = LogisticRegression(solver = "liblinear", n_jobs = 1)
        
MiniRocketLR = make_pipeline(trf, clf)

param_grid_MiniRocketLR = {
    "num_kernels" : [1000, 10000],
    'C': [0.01, 0.1, 1, 10, 100]
}

cv_results_mr, gs_objects_mr = get_cv_results(MiniRocketLR, param_grid_MiniRocketLR, df)

with open("cv_results/mr.pkl", "wb") as f:
    pickle.dump(cv_results_mr, f)

In [ ]:
itc = InceptionTimeClassifier(n_epochs = 50, batch_size = 16, metrics = ["f1_score"])

param_grid_itc = {
    "n_filters" : [6, 32, 64],
}

cv_results_itc, gs_objects_itc = get_cv_results(itc, param_grid_itc, df, n_jobs = 1)

with open("cv_results/itc.pkl", "wb") as f: 
    pickle.dump(cv_results_itc, f)

In [16]:
pd.DataFrame(cv_results_itc["operation"])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_filters,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,58.578768,1.090924,5.659585,0.440732,6,{'n_filters': 6},1.0,0.8,1.0,0.933333,0.094281,2
1,96.429901,1.789470,6.023480,0.471146,32,{'n_filters': 32},1.0,1.0,1.0,1.000000,0.000000,1
2,25.679865,0.263504,0.000000,0.000000,64,{'n_filters': 64},NaN,NaN,NaN,NaN,NaN,3


In [ ]:
mr_mean_ranks, best_mr = extract_mean_rank(cv_results_mr)
drcif_mean_ranks, best_drcif = extract_mean_rank(cv_results_drcif)
itc_mean_ranks, best_itc = extract_mean_rank(cv_results_itc)
tde_mean_ranks, best_tde = extract_mean_rank(cv_results_tde)

In [ ]:
rotf = RotationForest(n_estimators = 200, n_jobs = -1) 

In [ ]:
fp = Pipeline([
    (
        "transform",
        TSFreshFeatureExtractor(
            default_fc_parameters="efficient",
            show_warnings = False),
    ),
    (
        "classifier", 
         RotationForest(n_estimators = 50
     )
    ),
])

In [ ]:
trf = MiniRocketMultivariate(n_jobs = 1) 
clf = LogisticRegression(C = 0.01, penalty = "l2", solver = "liblinear", n_jobs = 1)
MiniRocketLR = make_pipeline(trf, clf)

In [ ]:
drcif = DrCIF(att_subsample_size = 5, time_limit_in_minutes = 10, n_jobs = -1)

In [ ]:
tde = TemporalDictionaryEnsemble(max_ensemble_size = 50, n_parameter_samples = 50, randomly_selected_params = 50, time_limit_in_minutes = 1, n_jobs = -1)

In [ ]:
itc = InceptionTimeClassifier(kernel_size = 10, n_filters = 6,n_epochs = 50, batch_size = 16)

In [ ]:
test_mr, mr_objects = get_test_results(MiniRocketLR)

In [ ]:
test_drcif, drcif_objects = get_test_results(drcif)

In [ ]:
test_tde, tde_objects = get_test_results(tde)

In [ ]:
test_itc, itc_objects = get_test_results(itc)

In [ ]:
results = pd.DataFrame(
    (
        test_mr["model_f1"], 
        test_drcif["model_f1"], 
        test_tde["model_f1"], 
        test_itc["model_f1"], 
        test_fp["model_f1"]
    )
)
results["model"] = ["MR", "DrCIF", "TDE", "ITC"]
results = results.set_index("model")
results = results.rename(columns = {0: "machine", 1 : "time", 2: "operation"})
results["avg"] = results.mean(axis = 1)

In [13]:
print(4096*(4096-1)/2)

8386560.0
